# Introduction to Dimension Reduction & Clustering

### ASI Data Science - 9th October 2017

#### Alexander Adam 

In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn import cluster, datasets
from sklearn.cluster  import KMeans
from sklearn.datasets import load_digits            
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.cluster.vq import kmeans

from IPython.display  import Image as display_image
%matplotlib inline  

### 1. Image Segmentation

In this first section, we introduce the k-means algorithm - the simplest clustering algorithm and its implementation in sklearn. As a simple application, we then show how k-means clustering can be used to segment an image by colour. By grouping similar (r,g,b) values together we should be able to extract different parts of the picture. We begin by loading an image below:

In [ ]:
img = sp.misc.imread('./Landscape_small.png')
display_image(filename='./Landscape_small.png', width = 380, height = 380)

The image above consists of (314 x 500) pixels. Each entry in the img array above encodes the colour values for each pixel. There are four values - the three primary (rgb) colours as well as an alpha (opacity) value. The opacity value is not relevant here so we'll discard it. We need to reshape the img array above in other to get it into a form that is suitable for the sklearn machine learning algorithms. These generally take two dimensional arrays where columns are regarded as features and rows as samples. We'll therefore reshape img into a three column array (r,g,b) with as many rows as there are pixels

In [ ]:
img = img[:, :, :3]

img_flat = sp.reshape(img, (img.shape[0]*img.shape[1],3)).astype(float)

print(img.shape, img_flat.shape)

K-means clustering in sklearn. Notice the similarity in syntax to the classifiers used in supervised learning. kmeans requires the number of clusters as input. The fit method calculates the centroids based on the input data, whilst the predict method assigns cluster membership to each point. Notice that once fitted, the predict method is able to take new data - so you can assign cluster membership to new points.

In [ ]:
# K-means in sklearn.

clf = KMeans(n_clusters = 3, init='k-means++')

clf.fit(img_flat.astype(float)) 

indices= clf.predict(img_flat)

In [ ]:
# Transform the image pack into 3D and pass back to plt.imshow()

plt.imshow(sp.reshape(indices, (img.shape[0], img.shape[1])))

The k-means algorithm requires the user to input the number of clusters. One issue is that in general we do not know a-priori what this value should be (unless we have application specific information). We should choose the number of clusters such that adding an additional cluster no longer gives 'better' modelling of the data. We can do this by plotting an objective function against the number of clusters and chosing the optimal value to be the point where the curve has a visible kink/elbow - indicating that adding additional clusters gives diminishing returns. Another different approach is to optimise a quantity known as the average silhouette coefficient. This is something you can read about here:

http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py

The code below shows how to generate such an 'elbow' plot using the average within cluster sum of squares as the objective function.

In [ ]:
centroids=[[] for i in range(1,10)]
euclidean_distances=[0 for i in range(len(img_flat))]
cluster_averaged_distances=[0 for i in range(1,10)]

for K in range(1,10):
    clf = KMeans(n_clusters = K, init='k-means++')
    clf.fit(img_flat.astype(float))
    
    indices= clf.predict(img_flat)    
    centroids= clf.cluster_centers_ 
    
    for j in range(len(img_flat)):
        euclidean_distances[j]=0
        cluster=indices[j]
        for k in range(3):
            euclidean_distances[j]+=(img_flat[j][k] - centroids[cluster][k])**2
        euclidean_distances[j]=(euclidean_distances[j])**0.5

    cluster_averaged_distances[K-1]=np.mean(euclidean_distances)
               
plt.plot(range(1,10),cluster_averaged_distances)
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
tt = plt.title('Example Elbow Curve')  

As an aside, the loops in the calculation above make it very inefficient. There is a faster 'black box' implementation available in scipy (that uses a very slightly different objective function). The scipy method holds the results in a somewhat different format. It stores the coordinates of all centroids together with the average distance of points from their closest centroid (which is what we're after)

In [ ]:
K = range(1,10)

KM = [kmeans(img_flat,k) for k in K] # Note the different call below to kmeans. This is scipy and not sklearn

distances=[KM[i][1] for i in range(len(KM))]

plt.plot(range(1,10),distances)
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
tt = plt.title('Elbow for K-Means clustering') 


#### Exercise: What's the optimal number of clusters above? Repeat the steps above with your favourite .png image, see if you can segment the colours that make up the image.



### 2. Digits and Dimension Reduction

We now look at the sklearn digits dataset. These are images of hand-written digits. A classic problem (and now benchmmark for models) in machine learning is whether it is possible to train a classifier to distinguish between these different digits. This dataset is labelled but here we'll look at the extent to which purely unsupervised methods are able to segment this dataset. Each digit is an (8x8) collection of pixels and there are a total of 1797 samples in the dataset. This particular digits dataset can be regarded as a low dimensional analogue of the famous MNIST dataset. You can apply the exact same methods used here to that much higher dimensional case.

In [ ]:
digits = load_digits()
digits.data.shape

X = np.vstack([digits.data[digits.target==i]
               for i in range(10)])
y = np.hstack([digits.target[digits.target==i]
               for i in range(10)])

In [ ]:
# Don't worry about the details of this plotting function. It's simply the show the handwritten digits to you.
nrows, ncols = 2, 5
plt.figure(figsize=(8,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(digits.images[i,...])
    plt.xticks([]); plt.yticks([])
    plt.title(digits.target[i])
plt.show()

In [ ]:
# For what follows it's useful to build a 2D scatter plot function that takes (data, labels) as arguments
import seaborn as sns 

def scatter(x, colors):
    palette = np.array(sns.color_palette("hls", 10))

    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])

    return f, ax, sc

Principal component analysis (PCA) in sklearn. The algorithm requires you to specify the number of principal components that you wish to retain. Note that PCA has both fit() and transform() methods. The former calculates the SVD of the input data and the necessary coordinate transformation to reduce the desired number of dimensions, whilst the latter actually applies the transformation. 

In [ ]:
pca = PCA(n_components=15)                           
X_pca = pca.fit_transform(X)
scatter(X_pca, y) 

As with clustering, it is important to have a mechanism to determine the correct number of principal components. In the case of PCA, the important quantity is the fraction of the total variance maintained as a function of the number of principal components retained. It is not uncommon to find in high dimensional dimension datasets that nearly all of the total variance can be retained whilst maintaining only a few principal components.

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum())
plt.ylabel('Fraction of Variance Retained')
plt.xlabel('Number of PCA Components')

#### Exercise: To what extent is PCA able to separate the digits dataset? To answer this question, first decide how many principal components you want to retain, then run k-means clustering on the result. Since the digits dataset is labelled, you can compare how closely your clustering labels match the true labels. You will find the following useful when comparing the labels:

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html

There are other dimensional reduction algorithms that can in certain situations perform better than PCA. The study of these falls under the topic of manifold learning. Such algorithms attempt to preserve non-linear information and structure from the original higher dimensional space. Many are for example sensitive to the topology of that space - such as whether it has holes. PCA in contrast is a purely linear operation - it knows nothing about this structure. One particularly powerful example of manifold learning is an algorithm called t-SNE (t-distributed stochastic neighbour embedding). The details underpinning it are fairly involved and we won't talk about them here. As you'll see if you do the exercise below, it is very dramatically able to separate the digits dataset. You can read more about the algorithm here:

http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html



#### Exercise II: To what extent is t-SNE able to separate the digits dataset? (The syntax for t-SNE is shown commented out below to help you). 

In [ ]:
# from sklearn.manifold import TSNE
# X_tsne = TSNE(random_state = 42).fit_transform(X)
# scatter(X_tsne, y) 

#### Exercise III: Use the cluster labels obtained from t-SNE + k-means as labels for supervised learning. Split the digits dataset (X,y) into a training and test set. Train a classifier (e.g. a random forest) to predict these cluster labels and validate it on the test set. For this digits dataset we actually have access to the 'true' labels so we can check how accurate the final results are. Remember that cluster label 0 is not necessarily equal to digit 0 so you'll want to work out what digit each cluster corresponds to.

### 3. Other Clustering Algorithms

Clustering is an enormous subject and we have barely scratched the surface this evening. k-means (and its variations) are often a very good place to start, but in many practical situations it will be insufficient. This final section shows you some examples of where k-means fails to cluster correctly and invites you to investigate the behaviour of various other clustering algorithms in sklearn. We compare their ability to cluster four different datasets. The plotting function below will also give you some indication of the relative speeds of the different clustering algorithms.

In [ ]:
# Generate four standard datasets for clustering. 

n_samples = 1500

noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None
colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
colors = np.hstack([colors] * 20)

In [ ]:
# A plotting function that will be used below. Do not worry about the details of this.

def PlotClustering(clustering_names, clustering_algorithms):
    
    plt.figure(figsize=(len(clustering_names) * 2 + 3, 9.5))

    plot_num = 1

    datasets = [noisy_circles, noisy_moons, blobs, no_structure]
    for i_dataset, dataset in enumerate(datasets):
        X, y = dataset
        X = StandardScaler().fit_transform(X) # NOTE: Normalise your data before clustering

        for name, algorithm in zip(clustering_names, clustering_algorithms):
            # Predict cluster memberships
            plt.subplot(4, len(clustering_algorithms), plot_num)
            if i_dataset == 0:
                plt.title(name, size=18)

            if algorithm is None:
                plt.scatter(X[:, 0], X[:, 1], color='gray', s=10)
            else:
                t0 = time.time()
                algorithm.fit(X)
                t1 = time.time()
                if hasattr(algorithm, 'labels_'):
                    y_pred = algorithm.labels_.astype(np.int)
                else:
                    y_pred = algorithm.predict(X)

                # Plot
                plt.scatter(X[:, 0], X[:, 1], color=colors[y_pred].tolist(), s=10)

                if hasattr(algorithm, 'cluster_centers_'):
                    centers = algorithm.cluster_centers_
                    center_colors = colors[:len(centers)]
                    plt.scatter(centers[:, 0], centers[:, 1], s=100, c=center_colors)
            plt.xlim(-2, 2)
            plt.ylim(-2, 2)
            plt.xticks(())
            plt.yticks(())
            if algorithm is not None:
                plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                         transform=plt.gca().transAxes, size=15,
                         horizontalalignment='right')
            plot_num += 1

    plt.show()    
    

#### Exercise: Test different clustering algorithms on the four datasets below. You'll find they have different behaviours.

In [ ]:
###### YOUR CODE GOES HERE ########
###################################

# FIND SOME CLUSTERING ALGORITHMS THAT BEST SEPARATE THIS DATA.
# You can see from what I've shown below that kmeans doesn't work very well here. 

###################################
###################################

from sklearn import cluster

kmeans = cluster.KMeans(n_clusters = 2)

names = ['Raw Data', 'kmeans']
algorithms = [None, kmeans] 

PlotClustering(names, algorithms)